# Gather Targeted Project Package Metrics

Project package metrics (PyPI, Conda, etc.) for software landscape analysis related to Cytomining ecosystem.

## Setup

Use of this notebook involves setup via https://github.com/ofek/pypinfo#installation. An environment variable is expected for pypinfo to work properly. For example: `export GOOGLE_APPLICATION_CREDENTIALS=/path/to/credentials.json`

In [14]:
import json
import os
import subprocess
from datetime import datetime

import awkward as ak
import condastats.cli as condastats_cli
import duckdb
import numpy as np
import pandas as pd
import pytz
from box import Box

In [15]:
# gather projects data
projects = Box.from_yaml(filename="data/target-projects.yaml").projects

# gather the lowercase loi focus project names from targets
loi_target_projects = [
    project["name"].lower()
    for project in projects.to_list()
    if "loi-focus" in project["category"]
]
loi_target_projects

['pycytominer', 'cytosnake', 'cytotable']

In [16]:
# create a str for targeting the specific projects
project_sql_str = ", ".join(["'" + project + "'" for project in loi_target_projects])

# filter results of github stats to find the project creation date for use in filtering below
with duckdb.connect() as ddb:
    loi_target_project_years = ddb.query(
        f"""
    SELECT
        ghstats."Project Name",
        ghstats."Date Created"
    FROM read_parquet('data/project-github-metrics.parquet') as ghstats
    WHERE LOWER(ghstats."Project Name") in ({project_sql_str})
    """,
    ).df()

loi_target_project_years

,Project Name,Date Created
0,pycytominer,2019-07-03 12:22:51-06:00
1,CytoSnake,2022-02-15 11:02:45-07:00
2,CytoTable,2022-09-08 09:46:25-06:00


In [17]:
# add a year created
loi_target_project_years["Date Created YYYY-MM"] = loi_target_project_years[
    "Date Created"
].dt.strftime("%Y-%m")
pkg_metrics = loi_target_project_years[
    ["Project Name", "Date Created YYYY-MM"]
].to_dict(orient="records")
pkg_metrics

[{'Project Name': 'pycytominer', 'Date Created YYYY-MM': '2019-07'},
 {'Project Name': 'CytoSnake', 'Date Created YYYY-MM': '2022-02'},
 {'Project Name': 'CytoTable', 'Date Created YYYY-MM': '2022-09'}]

In [18]:
# gather various PyPI metrics through pypinfo
pkg_metrics = [
    dict(
        project,
        **{
            # gather total downloads
            "pypi_downloads_total": json.loads(
                subprocess.run(
                    [
                        "pypinfo",
                        "--json",
                        project["Project Name"],
                    ],
                    capture_output=True,
                    check=True,
                ).stdout
            )["rows"],
            # gather downloads by year and month, ordered by month
            "pypi_downloads_by_month": json.loads(
                subprocess.run(
                    [
                        "pypinfo",
                        "--json",
                        "--start-date",
                        project["Date Created YYYY-MM"],
                        "--order",
                        "download_month",
                        project["Project Name"],
                        "month",
                    ],
                    capture_output=True,
                    check=True,
                ).stdout
            )["rows"],
            # gather downloads by python version
            "pypi_downloads_by_pyversion": json.loads(
                subprocess.run(
                    ["pypinfo", "--json", project["Project Name"], "pyversion"],
                    capture_output=True,
                    check=True,
                ).stdout
            )["rows"],
            # gather downloads by country
            "pypi_downloads_by_country": json.loads(
                subprocess.run(
                    ["pypinfo", "--json", project["Project Name"], "country"],
                    capture_output=True,
                    check=True,
                ).stdout
            )["rows"],
            # gather downloads by system and distro type
            "pypi_downloads_by_system_and_distro": json.loads(
                subprocess.run(
                    ["pypinfo", "--json", project["Project Name"], "system", "distro"],
                    capture_output=True,
                    check=True,
                ).stdout
            )["rows"],
        }
    )
    for project in pkg_metrics
]
ak.Array(pkg_metrics)

<Array [{'Project Name': ..., ...}, ..., {...}] type='3 * {"Project Name": ...'>

In [ ]:
# gather various conda metrics through condastats (seeks conda-forge and bioconda data)
pkg_metrics = [
    dict(
        project,
        **{
            # gather total downloads
            "conda_downloads_total": condastats_cli.overall(
                package=project["Project Name"],
                start_month=project["Date Created YYYY-MM"],
            ).to_dict(),
            # gather downloads by month
            "conda_downloads_by_month": condastats_cli.overall(
                package=project["Project Name"],
                start_month=project["Date Created YYYY-MM"],
                monthly=True,
            ).to_dict(),
            # gather downloads by python version
            "conda_downloads_by_pyversion": condastats_cli.pkg_python(
                package=project["Project Name"],
                start_month=project["Date Created YYYY-MM"],
            ).to_dict(),
            # gather downloads by version
            "conda_downloads_by_version": condastats_cli.pkg_version(
                package=project["Project Name"],
                start_month=project["Date Created YYYY-MM"],
            ).to_dict(),
            # gather downloads by system and distro type
            "conda_downloads_by_platform": condastats_cli.pkg_platform(
                package=project["Project Name"],
                start_month=project["Date Created YYYY-MM"],
            ).to_dict(),
        }
    )
    for project in pkg_metrics
]
ak.Array(pkg_metrics)